In [3]:
###########################################
# AMLO Pressconference Metadata scrap
###########################################

#save metadata as json
#get video ids
#!yt-dlp --flat-playlist --print "%(id)s" "https://www.youtube.com/playlist?list=PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA" > all_ids.txt
#!yt-dlp -j --no-download --cookies cookies.txt "https://www.youtube.com/playlist?list=PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA" > amlo_metadata_full.json



In [8]:
#check missed videos ids
import json


with open("all_ids.txt", "r") as f:
    all_ids = [line.strip() for line in f if line.strip()]


metadata = []
with open("amlo_metadata_full.json", "r") as f:
    for line in f:
        try:
            metadata.append(json.loads(line))
        except json.JSONDecodeError:
            continue  # fehlende zeile überspringen


scraped_ids = {entry['id'] for entry in metadata if 'id' in entry}


missing_ids = [vid for vid in all_ids if vid not in scraped_ids]

print(f"Gesamtanzahl IDs: {len(all_ids)}")
print(f"Erfolgreich gescrapete IDs: {len(scraped_ids)}")
print(f"Fehlende IDs (werden erneut gescraped): {len(missing_ids)}")

#save 
with open("missing_ids.txt", "w") as f:
    for vid in missing_ids:
        f.write(vid + "\n")

print("📄 missing_ids.txt wurde erstellt.")


✅ Gesamtanzahl IDs: 1389
✅ Erfolgreich gescrapete IDs: 377
❌ Fehlende IDs (werden erneut gescraped): 1012
📄 missing_ids.txt wurde erstellt.


In [13]:
import json
import subprocess

with open("missing_ids.txt", "r") as f:
    missing_ids = [line.strip() for line in f if line.strip()]


metadata_missing = []


for i, video_id in enumerate(missing_ids, 1):
    url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"[{i}/{len(missing_ids)}] Scraping {video_id} ...")
    
    try:
       
        result = subprocess.run(
            ["yt-dlp", "-j", "--no-download", "--cookies", "cookies.txt", url],
            capture_output=True,
            text=True,
            check=True
        )

        metadata = json.loads(result.stdout)
        metadata_missing.append(metadata)
        print(f"✅ Erfolgreich: {video_id}")
    
    except subprocess.CalledProcessError as e:
        print(f"❌ Fehler bei {video_id}: {e.stderr.strip()}")


with open("amlo_metadata_missing.json", "w") as f:
    json.dump(metadata_missing, f, indent=2)

print("✅ Alle fehlenden Metadaten gespeichert.")


[1/1012] Scraping Mg2tRUxh5cM ...
❌ Fehler bei Mg2tRUxh5cM: ERROR: [youtube] Mg2tRUxh5cM: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[2/1012] Scraping in6XfCiOieo ...
❌ Fehler bei in6XfCiOieo: WARNING: [youtube] in6XfCiOieo: nsig extraction failed: Some formats may be missing
         n = ppB_JkzXA7j90Zir8K ; player = https://www.youtube.com/s/player/20830619/player_ias.vflset/en_US/base.js
         n = lYWuWThJAUhxHLoWPY ; player = https://www.youtube.com/s/player/20830619/player_ias.vflset/en_US/base.js
ERROR: [youtube] in6XfCiOieo: This content isn't available, try again later.
[3/1012] Scraping eEF79Dcj_nU ...
❌ Fehler bei eEF79Dcj_nU: ERRO

In [14]:
import json

# Bestehende Metadaten laden
metadata_full = []
with open("amlo_metadata_full.json", "r") as f:
    for line in f:
        try:
            metadata_full.append(json.loads(line))
        except json.JSONDecodeError:
            continue

# Neue (nachgescrapete) Metadaten laden
with open("amlo_metadata_missing.json", "r") as f:
    metadata_missing = json.load(f)

# Kombinieren 
combined_metadata = {entry["id"]: entry for entry in metadata_full if "id" in entry}
for entry in metadata_missing:
    if "id" in entry:
        combined_metadata[entry["id"]] = entry  # überschreibt evtl. unvollständige alte Daten

# save again
with open("amlo_metadata_combined.json", "w") as f:
    for entry in combined_metadata.values():
        f.write(json.dumps(entry) + "\n")

print(f"✅ Kombinierte Datei mit {len(combined_metadata)} Einträgen gespeichert.")


✅ Kombinierte Datei mit 740 Einträgen gespeichert.


In [17]:

scraped_ids = set(combined_metadata.keys())

#compare again
with open("all_ids.txt", "r") as f:
    all_ids = [line.strip() for line in f if line.strip()]

still_missing = [vid for vid in all_ids if vid not in scraped_ids]

print(f"📊 Nach dem Merge fehlende IDs: {len(still_missing)}")

if still_missing:
    with open("still_missing_ids.txt", "w") as f:
        for vid in still_missing:
            f.write(vid + "\n")
    print("📄 still_missing_ids.txt wurde erstellt.")


📊 Nach dem Merge fehlende IDs: 649
📄 still_missing_ids.txt wurde erstellt.


In [18]:

with open("still_missing_ids.txt", "r") as f:
    still_missing_ids = [line.strip() for line in f if line.strip()]


metadata_still_missing = []


for i, video_id in enumerate(still_missing_ids, 1):
    url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"[{i}/{len(still_missing_ids)}] Scraping {video_id} ...")
    
    try:
        result = subprocess.run(
            ["yt-dlp", "-j", "--no-download", "--cookies", "cookies.txt", url],
            capture_output=True,
            text=True,
            check=True
        )
        metadata = json.loads(result.stdout)
        metadata_still_missing.append(metadata)
        print(f"✅ Erfolgreich: {video_id}")
    
    except subprocess.CalledProcessError as e:
        print(f"❌ Fehler bei {video_id}: {e.stderr.strip()}")

with open("amlo_metadata_still_missing.json", "w") as f:
    json.dump(metadata_still_missing, f, indent=2)

print("✅ Metadaten der letzten fehlenden Videos gespeichert.")

[1/649] Scraping Mg2tRUxh5cM ...
❌ Fehler bei Mg2tRUxh5cM: ERROR: [youtube] Mg2tRUxh5cM: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[2/649] Scraping in6XfCiOieo ...
✅ Erfolgreich: in6XfCiOieo
[3/649] Scraping eEF79Dcj_nU ...
✅ Erfolgreich: eEF79Dcj_nU
[4/649] Scraping KCTbAbNH2UQ ...
✅ Erfolgreich: KCTbAbNH2UQ
[5/649] Scraping 2ageOd2NtSQ ...
✅ Erfolgreich: 2ageOd2NtSQ
[6/649] Scraping pn1ae6a_ymE ...
✅ Erfolgreich: pn1ae6a_ymE
[7/649] Scraping s8ZJzF5bI7A ...
✅ Erfolgreich: s8ZJzF5bI7A
[8/649] Scraping Kbgeifv0q1A ...
✅ Erfolgreich: Kbgeifv0q1A
[9/649] Scraping hkVYRCpkbpo ...
✅ Erfolgreich: hkVYRCpkbpo
[10/649] Scraping slMGpOArZUs ...
✅ Erfo

In [21]:
# Lade die bisher kombinierten Metadaten (zeilenbasiert)
metadata_combined = []
with open("amlo_metadata_combined.json", "r") as f:
    for line in f:
        try:
            metadata_combined.append(json.loads(line))
        except json.JSONDecodeError:
            continue

# Lade die neuesten nachgescrapten Metadaten (JSON-Array)
with open("amlo_metadata_still_missing.json", "r") as f:
    metadata_still_missing = json.load(f)

# Kombiniere alle Metadaten und vermeide Duplikate
final_metadata = {entry["id"]: entry for entry in metadata_combined if "id" in entry}
for entry in metadata_still_missing:
    if "id" in entry:
        final_metadata[entry["id"]] = entry  # überschreibt ggf. ältere Daten

# Speichern als neue, finale Datei
with open("amlo_metadata_final.json", "w") as f:
    for entry in final_metadata.values():
        f.write(json.dumps(entry) + "\n")

print(f"✅ Finales Metadaten-File gespeichert mit {len(final_metadata)} eindeutigen Videos.")

# Lade alle ursprünglichen IDs
with open("all_ids.txt", "r") as f:
    all_ids = [line.strip() for line in f if line.strip()]

# Extrahiere gescrapete IDs aus finaler Datei
scraped_final_ids = set(final_metadata.keys())

# IDs, die auch jetzt noch fehlen
final_missing = [vid for vid in all_ids if vid not in scraped_final_ids]

print(f"🔍 Gesamtanzahl erwarteter IDs: {len(all_ids)}")
print(f"✅ Final erfolgreich gescrapete IDs: {len(scraped_final_ids)}")
print(f"❌ Noch fehlende IDs: {len(final_missing)}")

# Speichern der final fehlenden IDs (falls überhaupt noch welche)
if final_missing:
    with open("really_missing_ids.txt", "w") as f:
        for vid in final_missing:
            f.write(vid + "\n")
    print("📄 really_missing_ids.txt wurde erstellt.")
else:
    print("🎉 Alle Videos erfolgreich gescrapet!")

✅ Finales Metadaten-File gespeichert mit 1089 eindeutigen Videos.
🔍 Gesamtanzahl erwarteter IDs: 1389
✅ Final erfolgreich gescrapete IDs: 1089
❌ Noch fehlende IDs: 300
📄 really_missing_ids.txt wurde erstellt.


In [22]:
# Lade die wirklich fehlenden IDs
with open("really_missing_ids.txt", "r") as f:
    really_missing_ids = [line.strip() for line in f if line.strip()]

# Ergebnisliste für Metadaten
metadata_really_missing = []


for i, video_id in enumerate(really_missing_ids, 1):
    url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"[{i}/{len(really_missing_ids)}] Scraping {video_id} ...")
    
    try:
        result = subprocess.run(
            ["yt-dlp", "-j", "--no-download", "--cookies", "cookies.txt", url],
            capture_output=True,
            text=True,
            check=True
        )
        metadata = json.loads(result.stdout)
        metadata_really_missing.append(metadata)
        print(f"✅ Erfolgreich: {video_id}")
    
    except subprocess.CalledProcessError as e:
        print(f"❌ Fehler bei {video_id}: {e.stderr.strip()}")


with open("amlo_metadata_really_missing.json", "w") as f:
    json.dump(metadata_really_missing, f, indent=2)

print("✅ Letzte Metadaten gespeichert.")

[1/300] Scraping Mg2tRUxh5cM ...
❌ Fehler bei Mg2tRUxh5cM: ERROR: [youtube] Mg2tRUxh5cM: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[2/300] Scraping MdNE4sonZtY ...
✅ Erfolgreich: MdNE4sonZtY
[3/300] Scraping CPSiolnmuSo ...
✅ Erfolgreich: CPSiolnmuSo
[4/300] Scraping uvaTU0pPDpc ...
✅ Erfolgreich: uvaTU0pPDpc
[5/300] Scraping xQMurmJIlU0 ...
✅ Erfolgreich: xQMurmJIlU0
[6/300] Scraping Q4i3CBeRxKs ...
✅ Erfolgreich: Q4i3CBeRxKs
[7/300] Scraping Ifuyjm0kZyE ...
✅ Erfolgreich: Ifuyjm0kZyE
[8/300] Scraping k4W79WwM9Mw ...
✅ Erfolgreich: k4W79WwM9Mw
[9/300] Scraping P5u8pbDD9VI ...
✅ Erfolgreich: P5u8pbDD9VI
[10/300] Scraping ucQ2aSyrEOI ...
✅ Erfo

In [25]:

metadata_combined = []
with open("amlo_metadata_combined.json", "r") as f:
    for line in f:
        try:
            metadata_combined.append(json.loads(line))
        except json.JSONDecodeError:
            continue


with open("amlo_metadata_still_missing.json", "r") as f:
    metadata_still_missing = json.load(f)


with open("amlo_metadata_really_missing.json", "r") as f:
    metadata_really_missing = json.load(f)


final_metadata = {}

for entry in metadata_combined + metadata_still_missing + metadata_really_missing:
    if "id" in entry:
        final_metadata[entry["id"]] = entry


with open("amlo_metadata_final.json", "w") as f:
    for entry in final_metadata.values():
        f.write(json.dumps(entry) + "\n")

print(f"✅ Finales Metadatenfile erstellt mit {len(final_metadata)} eindeutigen Videos.")


✅ Finales Metadatenfile erstellt mit 1365 eindeutigen Videos.


In [26]:
# Öffne die JSON-Datei
with open('/Users/paulbachmann/Nextcloud/Amlo_paper/1_Scripts/amlo_metadata_final.json', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Parse jedes JSON-Objekt in ein Dictionary
videos = [json.loads(line) for line in lines]

# In DataFrame umwandeln
df = pd.DataFrame(videos)

# Checke die Spalten
df.columns.tolist()


['id',
 'title',
 'formats',
 'thumbnails',
 'thumbnail',
 'description',
 'channel_id',
 'channel_url',
 'duration',
 'view_count',
 'average_rating',
 'age_limit',
 'webpage_url',
 'categories',
 'tags',
 'playable_in_embed',
 'live_status',
 'media_type',
 'release_timestamp',
 '_format_sort_fields',
 'automatic_captions',
 'subtitles',
 'comment_count',
 'chapters',
 'heatmap',
 'like_count',
 'channel',
 'channel_follower_count',
 'channel_is_verified',
 'uploader',
 'uploader_id',
 'uploader_url',
 'upload_date',
 'timestamp',
 'availability',
 'original_url',
 'webpage_url_basename',
 'webpage_url_domain',
 'extractor',
 'extractor_key',
 'playlist_count',
 'playlist',
 'playlist_id',
 'playlist_title',
 'playlist_uploader',
 'playlist_uploader_id',
 'playlist_channel',
 'playlist_channel_id',
 'playlist_webpage_url',
 'n_entries',
 'playlist_index',
 '__last_playlist_index',
 'playlist_autonumber',
 'display_id',
 'fulltitle',
 'duration_string',
 'release_date',
 'release_year

In [27]:
df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y%m%d', errors='coerce')

print(df['upload_date'].dtype)
print(df['upload_date'].head())


datetime64[ns]
0   2024-09-30
1   2024-09-27
2   2024-09-26
3   2024-09-25
4   2024-09-24
Name: upload_date, dtype: datetime64[ns]


In [29]:
######################################
#only keep videos over 10 min
######################################

# read data 
with open('/Users/paulbachmann/Nextcloud/Amlo_paper/1_Scripts/amlo_metadata_final.json', 'r', encoding='utf-8') as f:
    lines = f.readlines()

videos = [json.loads(line) for line in lines]

#create df
df = pd.DataFrame(videos)

# needed collumms
df = df[['id', 'title', 'upload_date', 'duration', 'view_count', 'like_count']]

df['upload_date'] = pd.to_datetime(df['upload_date'], format='%Y%m%d')

#only videos with 10 or more min
df_long = df[df['duration'] >= 420]

print(f"Anzahl Videos >= 7 Minuten: {len(df_long)}")

df_long.head()

Anzahl Videos >= 7 Minuten: 1362


,id,title,upload_date,duration,view_count,like_count
0,Q5Z01pPwoEM,Última conferencia matutina del presidente And...,2024-09-30,7753,1814010,133867
1,kXHo_B3zn1I,Senado y congresos aprueban incorporar Guardia...,2024-09-27,11066,771696,66110
2,SroZUDpB3f8,Respaldo a presidenta electa en postura respec...,2024-09-26,10850,696026,59894
3,9W2VBIIjR7k,Ayotzinapa es expediente abierto; carta a fami...,2024-09-25,10952,622532,53420
4,S4QlSWnLT1w,Informe final El Pulso de la Salud. Conferenci...,2024-09-24,12207,559848,48876


In [30]:
#save dataframe as csv
df_long.to_csv('amlo_videos_meta.csv', index=False)


SyntaxError: invalid syntax (840458221.py, line 1)